In [1]:
!pip install mljar-supervised

     |████████████████████████████████| 101 kB 4.0 MB/s 
     |████████████████████████████████| 9.9 MB 8.2 MB/s 
     |████████████████████████████████| 27.4 MB 229 kB/s 
     |████████████████████████████████| 157.5 MB 61 kB/s 
     |████████████████████████████████| 1.7 MB 58.5 MB/s 
     |████████████████████████████████| 65.7 MB 58.3 MB/s 
     |████████████████████████████████| 21.8 MB 54.7 MB/s 
     |████████████████████████████████| 42 kB 868 kB/s 
     |████████████████████████████████| 319 kB 46.4 MB/s 
     |████████████████████████████████| 215 kB 47.1 MB/s 
     |████████████████████████████████| 293 kB 61.5 MB/s 
  Created wheel for mljar-supervised: filename=mljar_supervised-0.10.3-py3-none-any.whl size=134890 sha256=5c4796e54bd8296fa21fe76e8d7a23879b41b56e63945d0f26585bb1c561af01
  Stored in directory: /root/.cache/pip/wheels/a3/c8/5a/5c3617a8488b49cb9d5ae6b2d79b0e4e6ddcf381def60fde7f
  Created wheel for dtreeviz: filename=dtreeviz-1.0-py3-none-any.whl size=47219 sha25

In [2]:
!pip install fastparquet
!pip install python-snappy
!pip install pyarrow 
# !pip install pandas==1.1.5


     |████████████████████████████████| 120 kB 4.5 MB/s 
     |████████████████████████████████| 59 kB 4.2 MB/s 
  Created wheel for fastparquet: filename=fastparquet-0.5.0-cp37-cp37m-linux_x86_64.whl size=217575 sha256=d76d1b29344d722652fdac5575f586c559ed55a6f418a03f95190f5aa7188833
  Stored in directory: /root/.cache/pip/wheels/a4/c3/8a/eee13aaf7ac012dcf62c6b5905c74816e91518186de270e8c1
  Created wheel for thrift: filename=thrift-0.13.0-cp37-cp37m-linux_x86_64.whl size=409492 sha256=79f87d1337544f0e359acd7f438762b89845583c710de3db7831f6123c47fed9
  Stored in directory: /root/.cache/pip/wheels/79/35/5a/19f5dadf91f62bd783aaa8385f700de9bc14772e09ab0f006a
Successfully built fastparquet thrift
     |████████████████████████████████| 55 kB 2.0 MB/s 


In [3]:
import numpy as np
import pandas as pd
from supervised.automl import AutoML
from sklearn.preprocessing import LabelEncoder, StandardScaler

In [4]:
train = pd.read_csv("../input/tabular-playground-series-may-2021/train.csv")
test = pd.read_csv("../input/tabular-playground-series-may-2021/test.csv")

all_df = pd.concat([train, test], axis=0)
all_df = all_df.drop(['id', 'target'], axis=1)


In [5]:
lbe = LabelEncoder()
train['target'] = lbe.fit_transform(train['target'])

le = LabelEncoder()
for col in all_df.columns:
    all_df[col] = le.fit_transform(all_df[col])

for col in all_df.columns:
    all_df[col] = np.log1p(all_df[col])

In [6]:
train_df = all_df[:len(train)]
train_df['target'] = train['target']
test_df = all_df[len(train):]


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [7]:
train_df.head()

AttributeError: 'NotebookFormatter' object has no attribute 'get_result'

   feature_0  feature_1  feature_2  feature_3  feature_4  feature_5  \
0        0.0        0.0   0.693147        0.0   0.693147   0.000000   
1        0.0        0.0   0.000000        0.0   1.098612   0.693147   
2        0.0        0.0   0.000000        0.0   0.000000   0.000000   
3        0.0        0.0   0.000000        0.0   0.000000   0.000000   
4        0.0        0.0   0.000000        0.0   0.000000   0.000000   

   feature_6  feature_7  feature_8  feature_9  ...  feature_41  feature_42  \
0        0.0   0.000000        0.0   0.000000  ...         0.0    1.098612   
1        0.0   0.000000        0.0   0.000000  ...         0.0    1.098612   
2        0.0   0.000000        0.0   1.098612  ...         0.0    1.386294   
3        0.0   1.386294        0.0   0.000000  ...         0.0    1.098612   
4        0.0   0.000000        0.0   0.000000  ...         0.0    1.098612   

   feature_43  feature_44  feature_45  feature_46  feature_47  feature_48  \
0    3.091042         0.0  

In [8]:
x_cols = train_df.columns[:-1].tolist()
y_col = train_df.columns[-1]

In [9]:
automl = AutoML(
    mode="Optuna", 
    total_time_limit=14400,
    ml_task = 'multiclass_classification',
    stack_models = True,
    eval_metric= 'logloss',
    boost_on_errors=True,
    algorithms = ['LightGBM', 'Xgboost', 'CatBoost', 'Extra Trees'],
    optuna_time_budget = 1000
)
automl.fit(train[x_cols], train[y_col])

AutoML directory: AutoML_1
Expected computing time:
Total training time: Optuna + ML training = 18400 seconds
Total Optuna time: len(algorithms) * optuna_time_budget = 4000 seconds
Total ML model training time: 14400 seconds
The task is multiclass_classification with evaluation metric logloss
AutoML will use algorithms: ['LightGBM', 'Xgboost', 'CatBoost', 'Extra Trees']
AutoML will stack models
AutoML will ensemble availabe models
AutoML steps: ['simple_algorithms', 'default_algorithms', 'boost_on_errors', 'ensemble', 'stack', 'ensemble_stacked']
Skip simple_algorithms because no parameters were generated.
* Step default_algorithms will try to check up to 4 models


[I 2021-05-02 15:21:14,735] A new study created in memory with name: no-name-f22a3ddc-fe8a-4c25-bce7-20cdf9d7de20


Optuna optimizes LightGBM with time budget 1000 seconds eval_metric logloss (minimize)


[I 2021-05-02 15:21:38,263] Trial 0 finished with value: 1.0950673831044857 and parameters: {'learning_rate': 0.1, 'num_leaves': 725, 'lambda_l1': 0.003971722885615567, 'lambda_l2': 8.70060897787558e-05, 'feature_fraction': 0.8497510164532243, 'bagging_fraction': 0.8459830734829203, 'bagging_freq': 2, 'min_data_in_leaf': 27, 'extra_trees': True}. Best is trial 0 with value: 1.0950673831044857.
[I 2021-05-02 15:21:55,415] Trial 1 finished with value: 1.097634075948751 and parameters: {'learning_rate': 0.1, 'num_leaves': 1184, 'lambda_l1': 4.20004731740671, 'lambda_l2': 0.7645285581846926, 'feature_fraction': 0.5504720925486792, 'bagging_fraction': 0.6506965928763723, 'bagging_freq': 6, 'min_data_in_leaf': 81, 'extra_trees': False}. Best is trial 0 with value: 1.0950673831044857.
[I 2021-05-02 15:22:07,098] Trial 2 finished with value: 1.1043773857202874 and parameters: {'learning_rate': 0.1, 'num_leaves': 248, 'lambda_l1': 7.660681737059624e-08, 'lambda_l2': 3.191104605091362e-08, 'feat

There was an error during 1_Optuna_LightGBM training.
Please check AutoML_1/errors.md for details.


...

[I 2021-05-02 16:15:00,776] A new study created in memory with name: no-name-7947cbdb-c00c-4ed9-9c2a-703b8766a37b


Optuna optimizes Extra Trees with time budget 1000 seconds eval_metric logloss (minimize)


[I 2021-05-02 16:16:56,450] Trial 0 finished with value: 1.104745255196405 and parameters: {'criterion': 'entropy', 'max_depth': 21, 'min_samples_split': 40, 'min_samples_leaf': 54, 'max_features': 0.44335046161704333}. Best is trial 0 with value: 1.104745255196405.
[I 2021-05-02 16:18:22,674] Trial 1 finished with value: 1.1048938943980036 and parameters: {'criterion': 'gini', 'max_depth': 27, 'min_samples_split': 17, 'min_samples_leaf': 50, 'max_features': 0.27986667922981523}. Best is trial 0 with value: 1.104745255196405.
[I 2021-05-02 16:22:45,636] Trial 2 finished with value: 1.1020672431202136 and parameters: {'criterion': 'gini', 'max_depth': 32, 'min_samples_split': 45, 'min_samples_leaf': 31, 'max_features': 0.9585579601468681}. Best is trial 2 with value: 1.1020672431202136.
[I 2021-05-02 16:25:24,444] Trial 3 finished with value: 1.104518742566896 and parameters: {'criterion': 'gini', 'max_depth': 24, 'min_samples_split': 60, 'min_samples_leaf': 93, 'max_features': 0.686628

There was an error during 4_Optuna_ExtraTrees training.
Please check AutoML_1/errors.md for details.
Skip boost_on_errors because no parameters were generated.


AutoMLException: No models produced. 
Please check your data or submit a Github issue at https://github.com/mljar/mljar-supervised/issues/new.

In [10]:
preds = automl.predict_proba(test)

AutoMLException: Cannot load AutoML directory. expected str, bytes or os.PathLike object, not NoneType

In [11]:
sub = pd.read_csv("../input/tabular-playground-series-may-2021/sample_submission.csv")
sub[sub.columns[1:]] = preds

NameError: name 'preds' is not defined

In [12]:
sub.to_csv("mljar_automl_2.csv", index=False)

ImportError: cannot import name 'CompressionOptions' from 'pandas._typing' (/opt/conda/lib/python3.7/site-packages/pandas/_typing.py)

In [13]:
import pickle
outfile = open('preds_object', 'wb')
pickle.dump(preds, outfile)
outfile.close()

NameError: name 'preds' is not defined

In [14]:
automl.report()